# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando classificadores lineares, que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Nesse modelo vamos passar a considerar os textos das íntegras das peças de promoção de arquivamento.

**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados.

Vamos ler os textos das íntegras, limpá-los e associar ao dataframe com os dados de homologação.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"

In [4]:
# carga dos textos

from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('portuguese')
stop_words.extend(['n', 'nº', 'n°', 'n.º', ',', '.', '!', '?', ';', ':', '...', 'º', '–', '/', '(', ')'])

def get_text(file):
    with open(file, encoding='utf-8', errors='replace') as f:
        data=' '.join(line.strip() for line in f)
        f.close()
    
    return data.strip()

def get_text_non_stop_words(text):
    filtered_text = ''
    for w in word_tokenize(text, language='portuguese'):
        if w not in stop_words:
            filtered_text += w
            filtered_text += ' '            
    return filtered_text.strip()   
    

folder_integras = f'{PATH}/integras-textos'

texts = {}
for file in listdir(folder_integras):
    if isfile(join(folder_integras, file)):
        texts[file.split('.')[0]] = get_text_non_stop_words(get_text(join(folder_integras,file)))
    else:
        print('is not file', file)



In [5]:
# montando um DataFrame
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')
df_work = df_original.copy()
cols = ['id', 'homologado']
df_work.drop(inplace=True, columns=[col for col in df_work.columns if col not in cols])
df_work['peca_promocao'] = None
df_work.sample(5)

,homologado,id,peca_promocao
3201,1,83031205,None
6039,1,74959507,None
7310,1,69534991,None
2309,1,72443447,None
6734,0,70941373,None


In [6]:
# Associando o texto...
for key, text in zip(texts.keys(), texts.values()):
    if key != '' and text != '':
        df_work.loc[df_work.id == int(key), 'peca_promocao'] = text
        
df_work.sample(5)

,homologado,id,peca_promocao
7133,1,62431848,EXMO IC 1.14.000.000229/2014-57 Arquivamento 0...
3390,1,54520001,None
7432,1,69228152,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...
7173,1,69325522,Ofício Ccrim /96 NF 1.34.001.000248/2016-01 PR...
3845,1,79411018,PROCEDIMENTO PREPARATÓRIO PP Nº 1.25.006.00050...


In [7]:
df_work['peca_promocao'][165] # só pra confirmar que as stopwords foram retiradas

'_____________________________________________________________________________________________ Inquérito Civil 1.30.012.000817/2009-09 PROMOÇÃO DE ARQUIVAMENTO Trata-se inquérito civil IC instaurado investigar possíveis irregularidades instalações alojamento estudantil Universidade Federal Rio Janeiro UFRJ Feito relatado fls 461/464 Às fls 536-538 consta Recomendação dirigida reitoria UFRJ data 25 julho 2016 visando correção várias irregularidades alojamento estudantil constatadas durante instrução procedimento Desde então tem-se adotado medidas objetivando cumprimento todos itens recomendados Contudo início mês agosto 2017 noticiada ocorrência incêndio dependências alojamento situação agravou problemas existentes Alguns alunos chegaram ficar moradia alojados provisoriamente hotel pago UFRJ O ocorrido originou instauração novo procedimento investigatório âmbito desta procuradoria República 1.30.001.004190/2017-96 recentemente apensado autos Assim presente investigação atualmente possui

In [8]:
print(len(df_work[df_work['homologado'] == 1]), len(df_work[df_work['homologado'] == 0]))

8267 134


In [9]:
# removendo os sem textos de peça de promoção (problema nos dados)
print(len(df_work[df_work['peca_promocao'].isnull()]))
df_work.dropna(subset=['peca_promocao'], inplace=True)
df_work.reset_index(drop=True, inplace=True)
print(len(df_work[df_work['peca_promocao'].isnull()]))

1688
0


In [10]:
print(len(df_work[df_work['homologado'] == 1]), len(df_work[df_work['homologado'] == 0]))

6612 101


## Treinando um modelo de Bag of Words

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(df_work['peca_promocao'], df_work['homologado'], test_size=0.20, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5370,), (5370,), (1343,), (1343,))

In [14]:
print(X_train.iloc[0])

Cuida-se representação anônima versando sobre possíveis irr MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REPÚBLICA NO DISTRITO FEDERAL 4º OFÍCIO DE ATOS ADMINISTRATIVOS Inquérito Civil 1.16.000.002107/2017-23 Promoção Arquivamento 1469/2018 Trata-se Inquérito Civil instaurado partir representação Departamento Nacional Trânsito DENATRAN relatando possível negativa Banco Brasil S.A. aceitar documentos assinados aludido órgão mediante Sistema Eletrônico Informações SEI quais somente aceitos encaminhados assinatura manual Narra representante fls 4/4v “ DENATRAN utiliza Sistema Eletrônico Informações SEI conformidade Decreto 8.539 8 outubro 2015 ” tal motivo documentos naturalmente assinados digitalmente Assim menciona tais assinaturas seguem padrão Infraestrutura Chaves Públicas Brasileira ICP-Brasil força art 6º aludido Decreto “ A autoria autenticidade integridade documentos assinatura processos administrativos eletrônicos poderão ser obtidas meio certificado digital emitido âmbito Infraes

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words=None)
vect = cv.fit(X_train)
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

<5370x82501 sparse matrix of type '<class 'numpy.int64'>'
	with 2012769 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_vectorized, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
preds = lr.predict(vect.transform(X_test))

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def print_score(y, preds):
    print('Accuracy score:', accuracy_score(y, preds))
    print('Precision score: ', format(precision_score(y, preds)))
    print('Recall score: ', format(recall_score(y, preds)))
    print('F1 score: ', format(f1_score(y, preds)))
    print('AUC: ', roc_auc_score(y, preds))
    
print_score(y_test, preds)

Accuracy score: 0.9828741623231572
Precision score:  0.9850634802091113
Recall score:  0.9977307110438729
F1 score:  0.9913566328447951
AUC:  0.5226748793314603


In [21]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, preds)

array([[   1,   20],
       [   3, 1319]], dtype=int64)

In [22]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train_vectorized, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [23]:
preds = nb.predict(vect.transform(X_test))
print_score(y_test, preds)

Accuracy score: 0.9843633655994043
Precision score:  0.9843633655994043
Recall score:  1.0
F1 score:  0.9921200750469044
AUC:  0.5


In [24]:
confusion_matrix(y_test, preds)

array([[   0,   21],
       [   0, 1322]], dtype=int64)

### Bag of words com ngrams

In [25]:
vect = CountVectorizer(stop_words=None, ngram_range=(1,3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)

In [26]:
X_train_vectorized

<5370x3683507 sparse matrix of type '<class 'numpy.int64'>'
	with 8234254 stored elements in Compressed Sparse Row format>

In [27]:
lr = LogisticRegression()
lr.fit(X_train_vectorized, y_train)

preds = lr.predict(vect.transform(X_test))

print_score(y_test, preds)

Accuracy score: 0.9828741623231572
Precision score:  0.9843400447427293
Recall score:  0.9984871406959153
F1 score:  0.9913631242959068
AUC:  0.49924357034795763


In [28]:
confusion_matrix(y_test, preds)

array([[   0,   21],
       [   2, 1320]], dtype=int64)

In [29]:
nb = MultinomialNB()
nb.fit(X_train_vectorized, y_train)

preds = nb.predict(vect.transform(X_test))
print_score(y_test, preds)

Accuracy score: 0.9843633655994043
Precision score:  0.9843633655994043
Recall score:  1.0
F1 score:  0.9921200750469044
AUC:  0.5


In [30]:
confusion_matrix(y_test, preds)

array([[   0,   21],
       [   0, 1322]], dtype=int64)

In [32]:
# liberando memória

%xdel vect
%xdel X_train_vectorized

In [34]:
# Não estou conseguindo rodar essa célula: erro de memória (rodar no Paperspace)

#vect = CountVectorizer(stop_words=None, ngram_range=(1,5)).fit(X_train)
#X_train_vectorized = vect.transform(X_train)

#X_train_vectorized

## Tfidf

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(min_df=1, ngram_range=(1,4)).fit(X_train)
X_train_vectorized = vect.transform(X_train)

X_train_vectorized

MemoryError: 

In [48]:
lr = LogisticRegression()
lr.fit(X_train_vectorized, y_train)

preds = lr.predict(vect.transform(X_test))

print_score(y_test, preds)

Accuracy score: 0.9843633655994043
Precision score:  0.9843633655994043
Recall score:  1.0
F1 score:  0.9921200750469044
AUC:  0.5


In [49]:
confusion_matrix(y_test, preds)

array([[   0,   21],
       [   0, 1322]], dtype=int64)

In [50]:
nb = MultinomialNB()
nb.fit(X_train_vectorized, y_train)

preds = nb.predict(vect.transform(X_test))
print_score(y_test, preds)

Accuracy score: 0.9843633655994043
Precision score:  0.9843633655994043
Recall score:  1.0
F1 score:  0.9921200750469044
AUC:  0.5


In [51]:
confusion_matrix(y_test, preds)

array([[   0,   21],
       [   0, 1322]], dtype=int64)